# Testing for Ghosts in the Machine: Assuring 'Good Enough' Software Quality in AI-based systems<br>

## Artur Patoka
PyCon Italia<br>
Florence 2024

![prace_logos.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/prace_logos.png)

# Agenda

- Problem statement (the WHY)

- The challenges (the OH MY GOD 😱)

- The solutions (the relief 😌)

# Problem statement

![1.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/1.png)

![2.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/2.png)

![3.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/3.png)

![4.GIF](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/4.GIF)

# What about some real-life examples?

![air_canada.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/air_canada.png)

![chevy.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/chevy.png)

![dpd_1.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/dpd_1.png)
![dpd_2.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/dpd_2.png)

![fine.gif](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/fine.gif)

# My personal experience

## A chatbot assistant for logged-in customer with knowledge base access and several functions available

## `cli-gen` - a hobby project
![cli-gen_demo.gif](https://raw.githubusercontent.com/arturpat/cli-gen/main/cli-gen_demo.gif)
## https://github.com/arturpat/cli-gen

# The challenges
# (the OH MY GOD 😱)

## Everything is non-deterministic

>   `- 'kkkdf*!^#*8183@^%(<><br>}}'`
> 
>   `+ 'kkkdf*!^#*8183@^%(<>\n<br>}}'`

## Everything is slow

> Why don't you just execute tests calling the `openAI` API in parallel?

Well, rate limiting, mostly

## Everything changes on monthly or even weekly basis

## Deciding on scope
- Security (e.g. initial prompt protection)
- Performance
- Answers correctness
- Unwanted topics avoidance
- Testing how functions are called

## Requirements are simply not present ever

# How did we even get here?
![pepe_cry_hands.gif](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/pepe_cry_hands.gif)

![technology_s_curve.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/technology_s_curve.png)

![technology_s_curve_marked.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/technology_s_curve_marked.png)

![technology_s_curve_question.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/technology_s_curve_question.png)

## The solutions (the relief 😌)

## Good 'ol assert will do *sometimes*

```python
def test_ls(chat):
    command = chat.ask_gpt_code_only("list files in directory in the most simple way")
    assert command == "ls"


def test_rm(chat):
    command = chat.ask_gpt_code_only("remove file named test.txt")
    assert command == "rm test.txt"

# difflib.SequenceMatcher

```python
import difflib

def assert_match_or_almost_match(string1, string2):
    matcher = difflib.SequenceMatcher(None, string1, string2)
    assert matcher.ratio() > 0.93

# DeepEval
## https://github.com/confident-ai/deepeval

```python
from deepeval import assert_test
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase


def test_case():
    answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.5)
    test_case = LLMTestCase(
        input="What if these shoes don't fit?",
        # Replace with actual output from an LLM application
        actual_output="We offer a 30-day full refund at no "
                      "extra costs.",
        retrieval_context=["All customers are eligible for a 30 "
                           "day full refund at no extra costs."]
    )
    assert_test(test_case, [answer_relevancy_metric])

# Just add your openai api token to env vars and it simply works

## Back to DelayThePay insurance examples:

```python
def test_case():
    answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.5)
    test_case = LLMTestCase(
        input="How do I extend my car insurance?",
        actual_output="To extend your car insurance, log into the "
                      "portal (nopayanyway.delaythepay.com), "
                       "select “Log in”, then “My products”, find "
                       "your car there and click “extend”.",
        retrieval_context=["Extending the insurance can be done in"
                           " the customer portal. To get there "
                           "click Log In, find the car in My "
                           "Products and click Extend"]
    )
    assert_test(test_case, [answer_relevancy_metric])
    
    answer_relevancy_metric.measure(test_case)
    print(answer_relevancy_metric.score)
    print(answer_relevancy_metric.reason)


```
❯ pytest deepeval_showcase.py -s
=============== test session starts ==========================================
platform linux -- Python 3.11.9, pytest-8.2.0, pluggy-1.5.0
rootdir: /home/artur
plugins: xdist-3.6.1, deepeval-0.21.42, repeat-0.9.3, anyio-4.3.0
collected 1 item                                                                                                                                                                                                                     

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)... 
Done! (3.75s)
1.0
The score is 1.00 because the response is perfectly relevant and directly addresses the question about 
extending car insurance. Great job!
Running teardown with pytest sessionfinish....


===================== 1 passed in 9.37s ======================================

```python
def test_case():
    answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.5)
    test_case = LLMTestCase(
        input="How do I extend my car insurance?",
        # Replace this with the actual output from your LLM application
        actual_output="I don’t know how to help with that.",
        retrieval_context=["Extending the insurance can be done in"
                           " the customer portal. To get there "
                           "click Log In, find the car in My "
                           "Products and click Extend"]
    )

    answer_relevancy_metric.measure(test_case)
    print(answer_relevancy_metric.score, answer_relevancy_metric.reason)
    assert_test(test_case, [answer_relevancy_metric])

```
❯ pytest deepeval_showcase.py -s
=============== test session starts ==========================================
platform linux -- Python 3.11.9, pytest-8.2.0, pluggy-1.5.0
rootdir: /home/artur
plugins: xdist-3.6.1, deepeval-0.21.42, repeat-0.9.3, anyio-4.3.0
collected 1 item                                                                                                                                                                                                                     

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)... 
Done! (3.75s)
0.0
The score is 0.00 because the statement "I don’t know how to help with that." does not offer any relevant 
information or steps on how to extend car insurance.
Running teardown with pytest sessionfinish....
===================== FAILURES ======================================

```python
def test_case():
    answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.5)
    test_case = LLMTestCase(
        input="How do I extend my car insurance?",
        # Replace this with the actual output from your LLM application
        actual_output="You can extend your insurance in our "
                      "insurance portal.",
        retrieval_context=["Extending the insurance can be done in"
                           " the customer portal. To get there "
                           "click Log In, find the car in My "
                           "Products and click Extend"]
    )

    answer_relevancy_metric.measure(test_case)
    print(answer_relevancy_metric.score, answer_relevancy_metric.reason)
    assert_test(test_case, [answer_relevancy_metric])

```
❯ pytest deepeval_showcase.py -s
=============== test session starts ==========================================
platform linux -- Python 3.11.9, pytest-8.2.0, pluggy-1.5.0
rootdir: /home/artur
plugins: xdist-3.6.1, deepeval-0.21.42, repeat-0.9.3, anyio-4.3.0
collected 1 item                                                                                                                                                                                                                     

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)... 
Done! (3.75s)
1.0
The score is 1.00 because the response perfectly addresses the question about extending car 
insurance with no irrelevant statements. Great job!

Running teardown with pytest sessionfinish....
===================== FAILURES ======================================


In [ ]:
from deepeval import assert_test
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    SummarizationMetric,
)
from deepeval.test_case import LLMTestCase

```python
def test_case():
    answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.5)
    answer_faithfulness_metric = FaithfulnessMetric(threshold=0.5)
    contextual_precision_metric = SummarizationMetric(threshold=0.5)
    test_case = LLMTestCase(
        input="How do I extend my car insurance?",
        # Replace this with the actual output from your LLM application
        actual_output="You can extend your insurance in our "
                       "insurance portal.",
        retrieval_context=["Extending the insurance can be done in"
                           " the customer portal. To get there "
                           "click Log In, find the car in My "
                           "Products and click Extend"]
    )

    answer_relevancy_metric.measure(test_case)
    answer_faithfulness_metric.measure(test_case)
    contextual_precision_metric.measure(test_case)

```
❯ pytest deepeval_showcase.py -s
=============== test session starts ==========================================
platform linux -- Python 3.11.9, pytest-8.2.0, pluggy-1.5.0
rootdir: /home/artur
plugins: xdist-3.6.1, deepeval-0.21.42, repeat-0.9.3, anyio-4.3.0
collected 1 item                                                                                                                                                                                                                     

Relevancy score: 1.0, reason: The score is 1.00 because the output was perfectly relevant to the question about 
extending car insurance and contained no irrelevant statements. Great job!
Faithfulness score: 0.0, reason: The score is 0.00 because the actual output incorrectly states that insurance 
can be extended in 'our insurance portal', whereas the retrieval context specifies that it should be done in 
the 'customer portal'.
Context Precision score: 0.0, reason: The score is 0.00 because the summary includes extra information about 
extending insurance and an insurance portal that is not mentioned in the original text, and it fails to answer 
specific questions about the nature of the insurance discussed in the original text.
✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)... Done! (6.34s)
✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)... Done! (7.15s)    
✨ You're running DeepEval's latest Summarization Metric! (using gpt-4o, strict=False, async_mode=True)... Done! (7.15s)   

Running teardown with pytest sessionfinish....
===================== FAILURES ======================================

...
```python
actual_output="The fall of the Roman Empire is a complex "
              "and multifaceted event that historians and "
              "scholars continue to study and debate. "
              "Several factors contributed to its decline, "
              "and it's important to consider both the "
              "Western Roman Empire, which fell in 476 AD, "
              "and the Eastern Roman Empire or Byzantine "
              "Empire, which lasted until 1453.",
```
...

In [ ]:
```
❯ pytest deepeval_showcase.py -s
=============== test session starts ==========================================
platform linux -- Python 3.11.9, pytest-8.2.0, pluggy-1.5.0
rootdir: /home/artur
plugins: xdist-3.6.1, deepeval-0.21.42, repeat-0.9.3, anyio-4.3.0
collected 1 item                                                                                                                                                                                                                     

Relevancy score: 0.0, reason: The score is 0.00 because the statements about the fall and decline of the 
Roman Empire are completely irrelevant to the input, which specifically asks about extending car insurance.
Faithfulness score: 1.0, reason: The score is 1.00 because there are no contradictions. Great job maintaining 
perfect alignment with the retrieval context! Keep up the excellent work! 🌟
Context Precision score: 0.0, reason: The score is 0.00 because the summary introduces extra information 
not present in the original text and fails to address questions that the original text can answer, indicating 
a significant deviation from the original content.
  ✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)... Done! (10.97s)
  ✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)... Done! (4.72s)     
  ✨ You're running DeepEval's latest Summarization Metric! (using gpt-4o, strict=False, async_mode=True)... Done! (8.86s)    
 

Running teardown with pytest sessionfinish....
===================== FAILURES ======================================

## Sources
- `cli-gen` project: https://github.com/arturpat/cli-gen
- `DeepEval` project: https://github.com/confident-ai/deepeval
- Air Canada lawsuit: https://www.theguardian.com/world/2024/feb/16/air-canada-chatbot-lawsuit
- Chevey Tachoe for $1: https://twitter.com/ChrisJBakke/status/1736533308849443121
- DPD swear-bot: https://www.bbc.com/news/technology-68025677
- OpenAI rate limiting: https://platform.openai.com/docs/guides/rate-limits

![pres_qr.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/pres_qr.png)